# Реализация модели двустороннего защищенного чата в мессенджере по спецификации Signal

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# КГПСЧ
from Crypto.Random import get_random_bytes, random

# HKDF
from Crypto.Protocol.KDF import HKDF

# Вспомогательные радости
from collections import namedtuple
import pickle
import binascii

----

In [4]:
def as_hex(blob: bytes):
    """
    Перевод блоба в шестнадцатиричное представление
    """
    
    return binascii.hexlify(blob).decode()

----
## Реализация оберток над криптографическими алгоритмами и примитивами

In [5]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes, associated_data: bytes = b''):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = gost3412.GOST3412Kuznechik(self._key)
        cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, associated_data)
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes, associated_data: bytes = b''):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        try:
            encrypter = gost3412.GOST3412Kuznechik(self._key)
            cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())

            ct_with_tag = ciphertext + tag

            return cipher.open(nonce, ct_with_tag, associated_data)
        except ValueError as e:
            return None
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

---

In [6]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [7]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [8]:
class Curve(object):
    """
    Обертка над эллиптической кривой. Для простоты это одна из кривых,
    определенных в ГОСТ 34.10
    """
    
    CURVE = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        
        
    @staticmethod
    def generate_private_key():
        """
        Генерирование закрытого ключа
        """
        
        return gost3410.prv_unmarshal(get_random_bytes(64))
    
    
    @staticmethod
    def get_public_key(private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return gost3410.public_key(Curve.CURVE, private_key)
        
        
    @staticmethod
    def multiply_by_scalar(scalar, point):
        """
        Умножение точки эллиптической кривой на скаляр 
        """

        return Curve.CURVE.exp(scalar, *point)

    
    @staticmethod
    def add_points(lhs, rhs):
        """
        Сложение двух точек эллиптической кривой
        """

        return Curve.CURVE._add(*lhs, *rhs)

----
## Элементы протоколов из спецификации Signal

In [9]:
class KdfChainInternal(object):
    """
    Вспомогательный класс, описывающий KDF-цепочку
    """
    
    def __init__(self, key, key_length):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        self._key_length = key_length
        self._kdf_key = key
    
    
    def reset(self, key):
        """
        Сброс состояния, запись нового ключа в качестве
        первоначального
        """
        
        self._kdf_key = key
        
        
    def _kdf_update(self, data):
        self._kdf_key, message_key = HKDF(self._kdf_key, self._key_length, 
                                          data, Streebog, 2)
        return message_key

In [10]:
class KdfChain(KdfChainInternal):
    """
    Класс, описывающий KDF-цепочку, которая генерирует
    новые ключи только на основе своего состояния
    https://signal.org/docs/specifications/doubleratchet/#kdf-chains
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(KdfChain, self).__init__(key, key_length)
        
        
    def step(self):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(b'\x00' * 32)

In [11]:
class SymmetricRatchet(KdfChainInternal):
    """
    Симметричный храповик
    Новые ключи вырабатываются на основе выработанного 
    секрета Диффи-Хеллмана
    https://signal.org/docs/specifications/doubleratchet/#symmetric-key-ratchet
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(SymmetricRatchet, self).__init__(key, key_length)
        
        
    def step(self, serialized_dh_key):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(serialized_dh_key)

-----

In [12]:
class DiffieHellmanRatchet(object):
    """
    DH-храповик
    https://signal.org/docs/specifications/doubleratchet/#diffie-hellman-ratchet
    """
    
    PRIVATE = 0
    PUBLIC  = 1
    
    
    def __init__(self):
        """
        Инициализация, выработка начальной ключевой пары
        """
        
        self._current_foreign_public_key = None
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
    
    def is_uninitialized(self):
        """
        Проверка того, что открытый ключ другой стороны отсутствует
        """
        
        return self._current_foreign_public_key is None
    
    
    def initialize(self, public_key):
        """
        Инициализация открытого ключа другой стороны
        """
        
        self._current_foreign_public_key = public_key
        
        
    def is_same_public_key(self, public_key):
        """
        Проверка равенства открытых ключей
        """
        
        if self._current_foreign_public_key is None:
            return False
        
        return public_key == self._current_foreign_public_key
    
    
    def step(self, public_key=None):
        """
        Выработка общего секрета Диффи-Хеллмана
        """
        
        if self._current_key_pair is None:
            raise RuntimeError('Uninitialized DH Ratchet used to produce a shared secret')
        
        if public_key is not None:
            self._current_foreign_public_key = public_key
            
        return Curve.multiply_by_scalar(self._current_key_pair[DiffieHellmanRatchet.PRIVATE], 
                                        self._current_foreign_public_key)
    
    
    def generate_new_key_pair(self):
        """
        Генерирование новой ключевой пары
        """
        
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._current_key_pair[DiffieHellmanRatchet.PUBLIC]
    
    
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return private_key, public_key

----

In [13]:
#
# Описание сути следует смотреть здесь:
# https://signal.org/docs/specifications/x3dh/#publishing-keys
#

KeyBundle = namedtuple('KeyBundle', ['IK',    # Ключ-идентификатор (Identity key)
                                     'SPK',   # Подписанный предключ (Signed prekey)
                                     'Sig',   # Подпись предключа (Prekey signature)
                                     'OPKs',  # Коллекция одноразовых предключей (One-time prekeys)
                                    ])

In [14]:
Message = namedtuple('Message', ['public_key',  # Открытый ключ DH отправляющего
                                 'id',          # Идентитфикатор сообщения
                                 'data',        # Данные сообщения
                                ])

----
## Участники протоколов из спецификации Signal

In [15]:
class SignalServer(object):
    """
    Класс сервера Signal
    """
    
    def __init__(self):
        """
        Инициализация
        """
        
        #
        # Коллекция ключевых данных клиентов, индексируемая по номеру телефона
        #
        
        self._key_bundles = {}
        
        #
        # Коллекция очередей сообщения, индексируемая по паре номеров, 
        # представляющих направление передачи
        #
        
        self._messages = {}
        
        
    def register(self, phone_number: str, keys: KeyBundle):
        """
        Регистрация пользователя по номеру телефона. Сохраняет набор ключей
        """
        
        if phone_number in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is already registered')
            
        trace('[SignalServer]', f'Registered client with phone number {phone_number}')
            
        self._key_bundles[phone_number] = keys
        
        
    def get_public_keys(self, phone_number: str):
        """
        Получение ключей пользователя по его номеру телефона
        """
        
        if phone_number not in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is not registered yet')
            
        #
        # Достану ключи
        #
        
        keys = self._key_bundles[phone_number]
        one_time_keys = keys.OPKs
        
        #
        # Возьму случайный из одноразовых ключей
        #
        
        idx = random.randint(0, len(one_time_keys) - 1)
        result = KeyBundle(keys.IK, keys.SPK, keys.Sig, (one_time_keys.pop(idx), idx))
        
        trace('[SignalServer]', f'Using OPK at {idx}')
        
        #
        # Сохраню обновленный пакет ключей (без одного одноразового)
        #
        
        self._key_bundles[phone_number] = KeyBundle(keys.IK, keys.SPK, keys.Sig, one_time_keys)
        
        return result
    
    
    def push_message(self, phone_from: str, phone_to: str, message: Message):
        """
        Помещение сообщения в очередь
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            self._messages[search_key] = []
            
        trace('[SignalServer]', f'Saved message with id {message.id} from {phone_from} to {phone_to}')
            
        self._messages[search_key].append(message)
        
        
    def peek_message(self, phone_from, phone_to):
        """
        Получение сообщения из очереди
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            return None
        
        if self._messages[search_key] == []:
            return None
        
        return self._messages[search_key].pop(0)

In [16]:
class SignalClient(object):
    """
    Класс клиента Signal
    """
    
    PRIVATE = 0
    PUBLIC  = 1
    
    X3DH_MSG_ID      = 'X3DH_MSG'
    DH_RATCHET_PK_ID = 'DH_RATCHET_PK'
    
    RECEIVED_FLAG     = 0
    DH_RATCHET        = 1
    ROOT_KDF_CHAIN    = 2
    SENDING_RATCHET   = 3
    RECEIVING_RATCHET = 4
    MSG_ID            = 5
    
    
    def __init__(self, phone_number, one_time_keys=10, *, fake_ik=False):
        """
        Инициализация, генерирование ключей
        """
        
        self._phone_number  = phone_number
        
        self._identity_key  = SignalClient._generate_key_pair()
        self._signed_prekey = SignalClient._generate_key_pair()
        self._ot_prekeys    = [SignalClient._generate_key_pair() for _ in range(one_time_keys)]
        
        digest = hash_digest(Streebog, pickle.dumps(self._signed_prekey[SignalClient.PUBLIC]))
        self._signature = gost3410.sign(Curve.CURVE, 
                                        self._identity_key[SignalClient.PRIVATE], 
                                        digest)
        
        if fake_ik:
            #
            # Демонстрация неудачного выполнения, заменю идентификационный ключ
            #
            
            self._identity_key  = SignalClient._generate_key_pair()
        
        self._sessions = {}
        
        trace('[SignalClient]', f'Identity key: {self._identity_key}')
        trace('[SignalClient]', f'Signed prekey: {self._signed_prekey}')
        trace('[SignalClient]', f'Signature: {as_hex(self._signature)}')
        trace('[SignalClient]', f'One time keys: {len(self._ot_prekeys)}')
        
        
    def send_message(self, phone_number: str, message: str, server: SignalServer):
        """
        Отправка сообщения, установление сессии, если нужно
        """
        
        if phone_number not in self._sessions:
            #
            # Сессии еще нет, поэтому установлю соединение
            #
            
            return self._x3dh_init_session(phone_number, message, server)
        
        #
        # Так как протокол X3DH асинхронный, то я прямо сразу и отправлю сообщение
        #
        
        if self._sessions[phone_number][SignalClient.RECEIVED_FLAG]:
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            self._sessions[phone_number][SignalClient.RECEIVED_FLAG] = False
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key   = self._sessions[phone_number][SignalClient.DH_RATCHET].step()
            
            trace('[SignalClient]', 'Root KDF chain step')
            send_key = self._sessions[phone_number][SignalClient.ROOT_KDF_CHAIN].step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Sending ratchet reset')
            self._sessions[phone_number][SignalClient.SENDING_RATCHET].reset(send_key)
            
        #
        # Вырабатывается ключ с помощью симметричного храповика для отправки
        # На ключе шифруется само сообщение
        #
        
        trace('[SignalClient]', 'Sending ratchet step')
        message_key = self._sessions[phone_number][SignalClient.SENDING_RATCHET].step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as encryption key')
        encrypted_message = KuznyechikMGM(message_key).encrypt(message.encode())
        
        message_id = self._sessions[phone_number][SignalClient.MSG_ID]
        public_key = self._sessions[phone_number][SignalClient.DH_RATCHET].public_key
        
        self._sessions[phone_number][SignalClient.MSG_ID] += 1
        
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     message_id, 
                                                                     encrypted_message))
        
        
    def receive_message(self, phone_number: str, server: SignalServer):
        """
        Прием сообщения, установление сессии, если надо
        """
        
        message = server.peek_message(phone_number, self.phone_number)
        if message is None:
            trace('[SignalClient]', f'No message for {self.phone_number} from {phone_number}')
            return
        
        public_key, message_id, data = message
        
        trace('[SignalClient]', f'Received message with id {message_id} from {phone_number}')
        
        if message_id == SignalClient.X3DH_MSG_ID:
            #
            # Это инициирующее сообщения для установки сессии
            #
            
            return self._x3dh_accept_session(phone_number, data, server)
        
        if message_id == SignalClient.DH_RATCHET_PK_ID:
            #
            # Это нам ключ открытый прислали для инициализации
            #
            
            if not self._sessions[phone_number][SignalClient.DH_RATCHET].is_uninitialized():
                raise RuntimeError(f'Cannot initialize DH ratchet for {self.phone_number} twice')
                
            return self._sessions[phone_number][SignalClient.DH_RATCHET].initialize(public_key)
        
        #
        # Во-первых, факт приема зафиксирую 
        #
        
        self._sessions[phone_number][SignalClient.RECEIVED_FLAG] = True
        
        #
        # Не сменился ли открытый ключ собеседника?
        #
        
        if not self._sessions[phone_number][SignalClient.DH_RATCHET].is_same_public_key(public_key):
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key = self._sessions[phone_number][SignalClient.DH_RATCHET].step(public_key)
            self._sessions[phone_number][SignalClient.DH_RATCHET].generate_new_key_pair()
            
            trace('[SignalClient]', 'Root KDF chain step')
            recv_key = self._sessions[phone_number][SignalClient.ROOT_KDF_CHAIN].step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Receiving ratchet reset')
            self._sessions[phone_number][SignalClient.RECEIVING_RATCHET].reset(recv_key)
            
        #
        # Делаю шаг храповика приема
        #
            
        trace('[SignalClient]', 'Receiving ratchet step')
        message_key = self._sessions[phone_number][SignalClient.RECEIVING_RATCHET].step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as decryption key')
        decrypted_message = KuznyechikMGM(message_key).decrypt(*data)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
            
        return decrypted_message.decode()
        
        
    @property
    def phone_number(self):
        """
        Получение номер телефона
        """
        
        return self._phone_number
    
    
    @property
    def public_keys(self):
        """
        Получение открытых ключей
        """
        
        return KeyBundle(self._identity_key[SignalClient.PUBLIC], 
                         self._signed_prekey[SignalClient.PUBLIC], 
                         self._signature, 
                         [prekey[SignalClient.PUBLIC] for prekey in self._ot_prekeys])
        
        
    def _x3dh_init_session(self, phone_number: str, message: str, server: SignalServer):
        IKb, SPKb, Sigb, (OPKb, idx) = server.get_public_keys(phone_number)
        
        #
        # Проверяю подпись
        #
        
        digest = hash_digest(Streebog, pickle.dumps(SPKb))
        if not gost3410.verify(Curve.CURVE, IKb, digest, Sigb):
            trace('[SignalClient]', f'Cannot verify signature of client with phone {phone_number}')
            return
        
        #
        # Эфемерный ключ
        #
        
        EKa = SignalClient._generate_key_pair()
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._identity_key, SPKb)
        DH2 = SignalClient._dh(EKa, IKb)
        DH3 = SignalClient._dh(EKa, SPKb)
        DH4 = SignalClient._dh(EKa, OPKb)
        
        #
        # Вырабатываю ключ и создаю сообщение
        # В качестве данных я отправлю шифртекст сообщения,
        # идентификационный ключ, эфемерный ключ и индекс
        # использованного одноразового ключа
        #
        # Протокол разрешает использовать выработанный ключ прямо
        # для зашифрования первого сообщения: An initial ciphertext 
        # encrypted with some AEAD encryption scheme [4] using AD 
        # as associated data and using an encryption key which is 
        # either SK or the output from some cryptographic PRF keyed by SK.
        #
        
        AD = pickle.dumps(self._identity_key[SignalClient.PUBLIC]) + pickle.dumps(IKb)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        #
        # А теперь требуется создать состояние
        #
        
        self._initialize_state(phone_number, root_key)
        
        encrypted_message = KuznyechikMGM(root_key).encrypt(message.encode(), AD)
        message = (encrypted_message, self._identity_key[SignalClient.PUBLIC], 
                   EKa[SignalClient.PUBLIC], idx)
        
        server.push_message(self.phone_number, 
                            phone_number, 
                            Message(None, SignalClient.X3DH_MSG_ID, message))
        
    
    def _x3dh_accept_session(self, phone_number: str, x3dh_parameters: Message, server: SignalServer):
        encrypred_message, IKa, EKa, idx = x3dh_parameters
        
        #
        # Получу нужный одноразовый ключ
        #
        
        OPKb = self._ot_prekeys.pop(idx)
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._signed_prekey, IKa)
        DH2 = SignalClient._dh(self._identity_key, EKa)
        DH3 = SignalClient._dh(self._signed_prekey, EKa)
        DH4 = SignalClient._dh(OPKb, EKa)
        
        #
        # Вырабатываю ключ и читаю сообщение
        #
        
        AD = pickle.dumps(IKa) + pickle.dumps(self._identity_key[SignalClient.PUBLIC])
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        decrypted_message = KuznyechikMGM(root_key).decrypt(*encrypred_message, AD)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
        
        #
        # Теперь нужно инициализировать состояние и отослать открытый ключ
        #
        
        self._initialize_state(phone_number, root_key)
        
        public_key = self._sessions[phone_number][SignalClient.DH_RATCHET].public_key
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     SignalClient.DH_RATCHET_PK_ID, 
                                                                     None))
        
        return decrypted_message.decode()
        
        
    def _initialize_state(self, phone_number: str, root_key):
        self._sessions[phone_number] = [True, 
                                        DiffieHellmanRatchet(), 
                                        SymmetricRatchet(root_key), 
                                        KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                        KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                        0,
                                       ]
        
        
    def _try_mimic(self, phone_number, session_phone_number, session):
        self._phone_number = phone_number
        self._sessions[session_phone_number] = session.copy()
        self._sessions[session_phone_number][SignalClient.DH_RATCHET].generate_new_key_pair()
        
        trace('[SignalClient]', f'User now tries to be a user with phone {phone_number}')
        
        
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return private_key, public_key
    
    
    @staticmethod
    def _dh(key_pair, public_key):
        return Curve.multiply_by_scalar(key_pair[SignalClient.PRIVATE], public_key)
    
    
    @staticmethod
    def _produce_root_key(DH1, DH2, DH3, DH4):
        kdf_data = pickle.dumps(DH1) + pickle.dumps(DH2) + pickle.dumps(DH3) + pickle.dumps(DH4)
        return HKDF(kdf_data, KuznyechikMGM.KEY_SIZE, b'\x00' * 16, Streebog, 1)

----
## Демонстрация удачного выполнения протоколов

In [17]:
# Создаю сервер
srv = SignalServer()

In [18]:
# Пользователь Алиса
alice = SignalClient('+7 921 059-87-72')

[TRACE] [SignalClient] Identity key: (2738252788752701490790337007600039613883618509497694861690705737596019264139837645488627302415097682783723179552705684407787458594703696315213007742440800, (11295717227674371776208866005731517639519887603227264606931960224527553344990457910065728836438038625045123388850819308169243960497262290307023138234528085, 3757648319258402651528856996364669577090078630620695543444694045149335937061078976537732841228585957665414723579728584302046840959324343628953684001556450))
[TRACE] [SignalClient] Signed prekey: (2281622363151828191032454034715289244836481841278881144728186531923653337986744429590690316409254732382111059277902534359885578016222688825969116353633254, (11654778021180341884136137287399054701417574647345847388618715255047506982715168401300202327430398716534195059899608536925741150795111396163925947293307702, 237477506101889912262337798372349909220185000662776389651561462653074707034852914458726932149246157067138657955608040225225447406000210833

In [19]:
# Пользователь Боб
bob = SignalClient('+7 991 174-51-34')

[TRACE] [SignalClient] Identity key: (1599062948693195505305608922402545660674362010315531767792793055319374506306036404119231610892131734181006760970064009368874983509034600137075654091765035, (1349167729826370987285850191406695238016279355930247676186117371496062792572966858446310942426244438927158887066373714162111201356979380415646279614437271, 3291061829247426211689884712848641790670550489337464081601005117734735225072374153917186314459132433986777065179660873441467967980575835228474627675176429))
[TRACE] [SignalClient] Signed prekey: (2871920340417375017639723544524733537904952111540151596900368058311154405119636959933802021881475085212348019240756691128805885767679790892762544727221096, (5846484737689517895356903807925356420065990456151238351050162794643244984151840169364632314849675577845462809321136711945679869886936616938249776884711495, 52450598914449048877128514763741100313044201673832546975387159337109679916923161428258595612412881929191611444667244991589405856915005487745

In [20]:
# Регистрирую всех 
srv.register(alice.phone_number, alice.public_keys)
srv.register(bob.phone_number, bob.public_keys)

[TRACE] [SignalServer] Registered client with phone number +7 921 059-87-72
[TRACE] [SignalServer] Registered client with phone number +7 991 174-51-34


In [21]:
# Алиса пытается пообщаться с Бобом
alice.send_message(bob.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 8
[TRACE] [SignalClient] Use 80049588000000000000008a41557153ac60e39ca618897d6caf42d184efeec919f67249f81557bcd392d771aeafcd61af8294dac7a1140e22f09cd64e7bdc29e158df3071694cee84d150acd7008a40e20792fb9837517cd7758b0a2bdbde30d9a0af9a8830d97076f771fa66a5f8ce11963bce40da21d19dd868c964dc28799f0da0f0d88beb3bfb28be6b9c00bf4786942e80049587000000000000008a409797f172c1ac5b10f4a3a6d937bff87a1d8544fd814184e8f9155eb60c54e1d94565973532fbfe7510f0f404bd53a947cdd7d413a00efd0268f559b63098c2198a40edbd500fba25201972ff599318af1b3810f44dee9338ab584a7755fb5cff88d40dc21c9e0665d156172e452bd9de08b9b74773e8bfa3b0a2fc61f1900860d63e86942e as associated data
[TRACE] [SignalClient] Use f2d45e9dc304d2ec1d7a8d409c21128e38a42f9df3f21841087ea943277c7f0e as root key
[TRACE] [SignalServer] Saved message with id X3DH_MSG from +7 921 059-87-72 to +7 991 174-51-34


In [22]:
# Боб полчает сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049588000000000000008a41557153ac60e39ca618897d6caf42d184efeec919f67249f81557bcd392d771aeafcd61af8294dac7a1140e22f09cd64e7bdc29e158df3071694cee84d150acd7008a40e20792fb9837517cd7758b0a2bdbde30d9a0af9a8830d97076f771fa66a5f8ce11963bce40da21d19dd868c964dc28799f0da0f0d88beb3bfb28be6b9c00bf4786942e80049587000000000000008a409797f172c1ac5b10f4a3a6d937bff87a1d8544fd814184e8f9155eb60c54e1d94565973532fbfe7510f0f404bd53a947cdd7d413a00efd0268f559b63098c2198a40edbd500fba25201972ff599318af1b3810f44dee9338ab584a7755fb5cff88d40dc21c9e0665d156172e452bd9de08b9b74773e8bfa3b0a2fc61f1900860d63e86942e as associated data
[TRACE] [SignalClient] Use f2d45e9dc304d2ec1d7a8d409c21128e38a42f9df3f21841087ea943277c7f0e as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 991 174-51-34 to +7 921 059-87-72


'Hi!'

In [23]:
# Это небольшой костыль, тут я получаю инициализирующее 
# сообщение с открытым ключом Боба
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id DH_RATCHET_PK from +7 991 174-51-34


In [24]:
# Алиса отсылает сообщение
alice.send_message(bob.phone_number, "What's up?", srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 3ef4a8f8aba29f7be59826488a0c5aea543e94555cb10b1f1945e35657c22fd3 as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 921 059-87-72 to +7 991 174-51-34


In [25]:
# Боб принимает и расшифровывает
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 3ef4a8f8aba29f7be59826488a0c5aea543e94555cb10b1f1945e35657c22fd3 as decryption key


"What's up?"

In [26]:
# Можно послать несколько сообщений подряд
bob.send_message(alice.phone_number, 'Some dummy phrase just to continue the dialog', srv)
bob.send_message(alice.phone_number, 'And another one', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 6892ddd839ce9a5d01368ffa06f17897474bc18a42afb4d56993b63401704ee4 as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 991 174-51-34 to +7 921 059-87-72
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 115b0c7b3dc17914bcaafa247968328c6749ed1f637b66434a61da572f0e67ca as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 991 174-51-34 to +7 921 059-87-72


In [27]:
# И получить оба. Одно...
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 6892ddd839ce9a5d01368ffa06f17897474bc18a42afb4d56993b63401704ee4 as decryption key


'Some dummy phrase just to continue the dialog'

In [28]:
# ... за другим
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 991 174-51-34
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 115b0c7b3dc17914bcaafa247968328c6749ed1f637b66434a61da572f0e67ca as decryption key


'And another one'

In [29]:
# Можно снова сменить направление отправки и 
# посмотреть, что произошел шаг DH-храповика
alice.send_message(bob.phone_number, 'Yet another DH ratchet step demo', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 981dda9605f86f0d5d2a033a7d9afae4f51d67d13f032590cb7b52dd17aab423 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 921 059-87-72 to +7 991 174-51-34


In [30]:
# А на другой строне надо получить сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 981dda9605f86f0d5d2a033a7d9afae4f51d67d13f032590cb7b52dd17aab423 as decryption key


'Yet another DH ratchet step demo'

In [31]:
# Можно третьего пользователя создать
carol = SignalClient('+7 970 505-44-25')
srv.register(carol.phone_number, carol.public_keys)

[TRACE] [SignalClient] Identity key: (2117920883207471110582081875611923156726293990690655695223603010335696788068733832004075298380611399768374556540614484650568555475588353314217357710989677, (11809099423182655600377408109291334777220536256407316138763814634252652759816438904240821898126314632845097774295159080233284400765313018547996242307953321, 10903507346141408179816909557787081593382524211241096273166742612088682927028855451766652422786376645495395248205125955173089780501016222513680477880933197))
[TRACE] [SignalClient] Signed prekey: (4059114539522998607417168955427758534009539232263002237776884835226448415663123310696236682674242858941665767458189056968874402465989466587546036662658465, (4327581356860228111414878230754518197650852500630774771362913995388504163220970110139306941920986779608925844921584924805122569205872892182396906309253660, 124676367725897382138342573429171756193451378682512979685611574567710503445511679713201177030516824799972679572588212337964016885596566132

In [32]:
alice.send_message(bob.phone_number, "I'm making new friends", srv)
alice.send_message(carol.phone_number, 'Hi, Carol!', srv)

[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 089812b2fee62f75fcfeeb5b78d07f15da14a0ae574b9661990641a3839a1929 as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 921 059-87-72 to +7 991 174-51-34
[TRACE] [SignalServer] Using OPK at 8
[TRACE] [SignalClient] Use 80049588000000000000008a41557153ac60e39ca618897d6caf42d184efeec919f67249f81557bcd392d771aeafcd61af8294dac7a1140e22f09cd64e7bdc29e158df3071694cee84d150acd7008a40e20792fb9837517cd7758b0a2bdbde30d9a0af9a8830d97076f771fa66a5f8ce11963bce40da21d19dd868c964dc28799f0da0f0d88beb3bfb28be6b9c00bf4786942e80049589000000000000008a41a9b2ab64faa634607f1e7c4e9d05289c2d96b1ef659d92f6e57bb47d52783c87f8e3b193c00e2fa979f88b47bae15faca66b437988de2a00208921ebf3ac79e1008a414d5bdd93d675a80f3450c74cbcae9d0e96135b765beb700855ceedb4d3464dde65b3b58a1733d4c5190a80f33394b0d721a7151688ada00ad2800ebfd53b2fd00086942e as associated data
[TRACE] [SignalClient] Use d8f46db580ad169c5c0687972edf2866c9adcef463eb1719a959b0a

In [33]:
carol.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049588000000000000008a41557153ac60e39ca618897d6caf42d184efeec919f67249f81557bcd392d771aeafcd61af8294dac7a1140e22f09cd64e7bdc29e158df3071694cee84d150acd7008a40e20792fb9837517cd7758b0a2bdbde30d9a0af9a8830d97076f771fa66a5f8ce11963bce40da21d19dd868c964dc28799f0da0f0d88beb3bfb28be6b9c00bf4786942e80049589000000000000008a41a9b2ab64faa634607f1e7c4e9d05289c2d96b1ef659d92f6e57bb47d52783c87f8e3b193c00e2fa979f88b47bae15faca66b437988de2a00208921ebf3ac79e1008a414d5bdd93d675a80f3450c74cbcae9d0e96135b765beb700855ceedb4d3464dde65b3b58a1733d4c5190a80f33394b0d721a7151688ada00ad2800ebfd53b2fd00086942e as associated data
[TRACE] [SignalClient] Use d8f46db580ad169c5c0687972edf2866c9adcef463eb1719a959b0a3f51cade3 as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 970 505-44-25 to +7 921 059-87-72


'Hi, Carol!'

In [34]:
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 921 059-87-72
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 089812b2fee62f75fcfeeb5b78d07f15da14a0ae574b9661990641a3839a1929 as decryption key


"I'm making new friends"

----
## Демонстрация неуспешного выполнения протоколов

In [35]:
# Пользователь, который как-то украл подпись и подписанный ключ
# Но его идентификационный ключ не позволяет проверить подпись
mallory = SignalClient('+1 202-918-2132', fake_ik=True)

[TRACE] [SignalClient] Identity key: (1200915346857794191820279703972097931858283076887019595525356638878465417742741465787143650647403175714056763890610383133143552021040364611875583140255243, (5803303135943748118619203399555352492152932495874236271654492552459278362315062187511274547779953330200358885397081726001706152505792697995559458322426708, 6574367360620898527413892307976069209233038033405867718028105693326584052005559643971451616432241903401284021912557177388721811279294603948721231409603131))
[TRACE] [SignalClient] Signed prekey: (10967598436658177290148141152047468050360834935784069666412941000509753773769284288174389409256992316546844387131931886969440730573137668333438612306603478, (2333461291373092857063819919710324524670539553786111195502843663112701863672136400475196810976312318178251263198405940168219744936736492214100689685364923, 2235575694395990164618991031938511962486932696102038957928606720861420879307705643087585371844311068628924027505304560230508783942560597687

In [36]:
# Надо зарегистрироваться
srv.register(mallory.phone_number, mallory.public_keys)

[TRACE] [SignalServer] Registered client with phone number +1 202-918-2132


In [37]:
# Алиса получает ключи с сервера и обнаруживает несоответствие
alice.send_message(mallory.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 6
[TRACE] [SignalClient] Cannot verify signature of client with phone +1 202-918-2132


In [38]:
# Еще один пользователь-враг -- Ева
eve = SignalClient('+1 539-567-2473')

[TRACE] [SignalClient] Identity key: (12022797398065306557755176856294478031265359122610219260565756963935711375100746190512786382206706695978204537470944261218344901788943874493686124343981258, (3815850571486648566052725503670241868868016091715739809591612158123579983392877515229193974822839843509167149625857549302485546451343311862783259785079105, 5874854738445235967450395796995505903122909255341200401753884601278941630364736527804282727646283599645983906951138588730409063167280741893366185352371659))
[TRACE] [SignalClient] Signed prekey: (7771842779491967795722490062754243431694771828514543206773230079366713729593961556635938975140556527770496937399930185553569950532992806277492933787792543, (1517507709472870130506685305238717391370017949743350562100587034924261293962001772295118950594862404664743656695161339932865224419500979135275062393824694, 9529618081590721923089391218348363645754280316241574463883406235537345936121674981443214246081676213479984956886577528159687171902537088186

In [39]:
# Она пытается прикинуться Алисой в разговоре с Бобом
eve._try_mimic(alice.phone_number, bob.phone_number, 
               alice._sessions[bob.phone_number])

[TRACE] [SignalClient] User now tries to be a user with phone +7 921 059-87-72


In [40]:
# Боб отправляет Алисе соообщение
bob.send_message(alice.phone_number, 'Hi!', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use f20f6bd5a31135e9544470e09de4974559d592b7bcc20574c9cc34fd385abf91 as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 991 174-51-34 to +7 921 059-87-72


In [41]:
# А вот получить его пытается Ева, но расшифровать не получится
eve.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use db025f64a0dca8ee5411cb5eea9728898a5a4ad3e68248f6cd652d849f4f201f as decryption key
[TRACE] [SignalClient] Cannot decrypt message
